In [1]:
import os
os.popen('ls').read()

'PW002044.owl\nSMP0000060\nconvert_pathbank.ipynb\npathbank_all_biopax.zip\n'

In [40]:
def xml_key(l):
    if l and l[:4] == '<bp:' and l[1] != '/' and l[1] != '?':
        return l.split()[0][4:]

with open('PW002044.owl') as owl:
    owl = [l.rstrip() for l in owl.readlines()]
xml = set([xml_key(l) for l in owl if xml_key(l)])
xml

{'BioSource',
 'BiochemicalPathwayStep',
 'BiochemicalReaction',
 'Catalysis',
 'CellularLocationVocabulary',
 'ChemicalStructure',
 'Complex',
 'Interaction',
 'Pathway',
 'PathwayStep',
 'Protein',
 'ProteinReference',
 'SmallMolecule',
 'SmallMoleculeReference',
 'Stoichiometry',
 'Transport',
 'UnificationXref'}

In [155]:
%reset -sf 
import xml.etree.ElementTree as ET
import networkx as nx

context = ET.iterparse('PW002044.owl', events=('end', ))
tree = [elem for event, elem in context]

for i in range(len(tree)):
    tree[i].set('pyindex', i)

child_to_parent, parent_to_children = {}, {} # direct descendants only
elem_data, IDs = {}, {}
for elem in tree:    
    elem_data[elem.get('pyindex')] = {}
    TAG = elem.tag
    if '{' == TAG[0]:
        TAG = TAG.split('}')[1]
    elem_data[elem.get('pyindex')]['tag'] = TAG
    
    for key, value in elem.items():
        KEY = key
        if '{' == key[0]:
            KEY = key.split('}')[1]
        if KEY == 'resource':
            elem_data[elem.get('pyindex')]['type'] = value[1:].split('/')[0]
        if KEY == 'ID':
            elem_data[elem.get('pyindex')]['type'] = value.split('/')[0]
            IDs[value] = elem.get('pyindex')
        elem_data[elem.get('pyindex')][KEY] =  value
    
    for child in elem:
        if len(list(child)) == 0:
            child_to_parent[child.get('pyindex')] = elem.get('pyindex')
            if elem.get('pyindex') not in parent_to_children:
                parent_to_children[elem.get('pyindex')] = []
            parent_to_children[elem.get('pyindex')].extend([str(child.get('pyindex'))])  
            
# Track tags
tags = {t: [] for t in set(elem_data[e]['tag'] for e in elem_data)}
for elem in elem_data:
    tags[elem_data[elem]['tag']].extend([elem_data[elem]['pyindex']])

# Add data to Graph object
G = nx.DiGraph()

G.add_nodes_from([(e, elem_data[IDs[e]]) for e in IDs])

# Add pathway order to Graph
for parent in tags['Pathway']:
    i = elem_data[parent]['pyindex']
    G.add_nodes_from([(i, elem_data[parent])])
    pathwayOrder = [c for c in parent_to_children[i] if elem_data[int(c)]['tag'] == 'pathwayOrder']
    for child in pathwayOrder:
        id = elem_data[int(child)]['resource'][1:]
        
        
# Add SmallMoleculeReference to Graph
for elem in tags['SmallMoleculeReference']:
    #print(elem_data[elem])
    
    
    break
    
#print(tags.keys())

    